In [46]:
import pickle
from sklearn.base import is_regressor
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [47]:
# 📌 Stap 1: Lees de datasets in
datasets = {
    "student-mat": pd.read_csv("C:/Users/xande/Desktop/bit-academy/Portefolio Project/Portfolio-Project/Data/student-mat.csv", sep=";"),
    "student-por": pd.read_csv("C:/Users/xande/Desktop/bit-academy/Portefolio Project/Portfolio-Project/Data/student-por.csv", sep=";")
}

target = 'G3'

In [48]:
# 📌 Stap 2: Preprocess data eerst! (X_train MOET beschikbaar zijn)
first_dataset = next(iter(datasets.values()))
first_dummies = pd.get_dummies(first_dataset, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    first_dummies.drop(columns=[target]), 
    first_dummies[target], 
    test_size=0.2, 
    random_state=42
)

In [49]:
# 📌 Stap 3: Model opslag / inladen
class Readwritemodel():
    def __init__(self, model=None, filenaam=None):
        self.filenaam = filenaam
        self.model = model  
    
    def check_input(self, var):
        if isinstance(var, str) and var.endswith(".sav"):
            return True
        else:
            print("Jouw filenaam moet een string zijn en eindigen met .sav")
            return False

    def opslaanmodel(self):
        if self.check_input(self.filenaam):
            pickle.dump(self.model, open(self.filenaam, 'wb'))
            print(f"Jouw model is opgeslagen onder de naam: {self.filenaam}")
        
    def inladenmodel(self):
        if self.check_input(self.filenaam):
            try:
                geladen_model = pickle.load(open(self.filenaam, 'rb'))
                if is_regressor(geladen_model):
                    print("Jouw model is opgehaald en je kan er nu mee voorspellen!")
                    return geladen_model
                else:
                    print("Dit is geen regressie model")
                    return None
            except FileNotFoundError:
                print("Het modelbestand kan niet gevonden worden. Er wordt een nieuw model getraind.")
                return None 

In [50]:
# 📌 Stap 4: Modelbeheer
func = Readwritemodel(filenaam="regressie.sav")
model = func.inladenmodel()

if model is None:
    print("Ik train een model...")
    model = GradientBoostingRegressor()
    model.fit(X_train, y_train)
    func.model = model
    func.opslaanmodel()
else:
    print("Je gebruikt nu een opgeslagen model")

Jouw model is opgehaald en je kan er nu mee voorspellen!
Je gebruikt nu een opgeslagen model


In [ ]:
# 📌 Stap 5: Loop door datasets en maak voorspellingen
for name, data in datasets.items():
    print(f"Verwerken van dataset: {name}")
    data_dummies = pd.get_dummies(data, drop_first=True)
    X = data_dummies.drop(columns=[target], errors="ignore")

    # ✅ FIX: Controleer of `X_train` beschikbaar is
    if 'X_train' not in globals():
        raise NameError("⚠️ FOUT: X_train is niet gedefinieerd!")

    # Zorg dat kolommen overeenkomen
    missing_cols = set(X_train.columns) - set(X.columns)
    for col in missing_cols:
        X[col] = 0
    
    X = X[X_train.columns]
    
    y_pred = model.predict(X)
    data['Predictions'] = y_pred
    
    output_filename = f"{name}_regression_predictions.csv"
    data.to_csv(output_filename, sep=";", index=False)
    print(f"Voorspellingen opgeslagen in {output_filename}")

Verwerken van dataset: student-mat
Voorspellingen opgeslagen in student-mat_regression_predictions.csv
Verwerken van dataset: student-por
Voorspellingen opgeslagen in student-por_regression_predictions.csv


In [52]:
# 📌 Stap 6: Evaluatie op de testset
mse = mean_squared_error(y_test, model.predict(X_test))
r2 = r2_score(y_test, model.predict(X_test))
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")

Mean Squared Error (MSE): 4.019045672650393
R-squared (R²): 0.8039972490622082
